In [84]:
#Importing necessary modules 
import random
from deap import base, creator, tools, algorithms
import numpy
import math

In [85]:
# Empezamos por declarar el fitness de cada individuo: Si es un problema
# de maximización el valor del atributo weights es 1, si por el contrario
# el problema es de minimización el valor es -1.
# Consideramos que nuestro objetivo es reducir al mínimo el número de errores
# que contenga el circuito.


In [113]:
creator.create('Fitness', base.Fitness, weights=(-1.0,))

D:\Program Files\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [114]:
#Consideramos los individuos del problema como las puertas lógicas.

In [115]:
creator.create('Individuo', list, fitness = creator.Fitness)

D:\Program Files\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individuo' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [26]:
# A continuación debemos de crear la caja de herramienta donde vamos a
# registrar los elementos necesarios para poder aplicar el algoritmo genético.

In [116]:
caja_de_herramientas = base.Toolbox()

In [117]:
#Declaramos las puertas lógicas
def puerta_or(a,b):
    return a+b

def puerta_and(a,b):
    return a*b

def puerta_not(a):
    if a==0:
        return 1
    if a==1:
        return 0

def puerta_nand(a,b):
    if a==0 and b==0:
        return 1
    elif a==0 and b==1:
        return 1
    elif a==1 and b==0:
        return 1
    elif a==1 and b==1:
        return 0

def puerta_xor(a,b):
    if a==1 and b==1:
        return 0
    return a+b

In [118]:
# Registramos una función gen que devuelve un número de 0 a 4, los cuales
# representan las distintas puertas anteriores puestas en orden.

In [119]:
caja_de_herramientas.register('gen', random.randint, 0, 5)

In [120]:
# Vamos a implementar la posibilidad de elegir libremente los valores del
# número de capas y de puertas lógicas

In [129]:
M = int(input("Numero de capas: "))
N = int(input("Numero de puertas: "))
#Semilla para la generación de números aleatorios
print("Circuito Inicial:")
random.seed(12345) 
for _ in range(M*N):
    print(caja_de_herramientas.gen())
    

Numero de capas: 3
Numero de puertas: 3
Circuito Inicial:
3
5
0
2
2
1
2
4
3


In [65]:
# A continuación llamamomos a la funcion individuo que nos genera un individuo
# con sus genes. Las conexiones máximas que puede tener una puerta son 3.

In [130]:
caja_de_herramientas.register('individuo', tools.initRepeat,
                              container=creator.Individuo, func=caja_de_herramientas.gen, n=3*(M*N))

In [131]:
random.seed(12345)
caja_de_herramientas.individuo()
# El primer gen se asocia al tipo de puerta de la capa, el segundo y el tercero
# se asocia al tipo de puerta del que recibe señal

[3,
 5,
 0,
 2,
 2,
 1,
 2,
 4,
 3,
 1,
 2,
 0,
 3,
 2,
 4,
 5,
 1,
 4,
 4,
 1,
 2,
 5,
 5,
 0,
 4,
 5,
 3]

In [132]:
# Consideramos una población como una lista de 10 individuos

In [133]:
caja_de_herramientas.register('población', tools.initRepeat,
                              container=list, func=caja_de_herramientas.individuo, n=10)

In [134]:
random.seed(12345)
caja_de_herramientas.población()

[[3,
  5,
  0,
  2,
  2,
  1,
  2,
  4,
  3,
  1,
  2,
  0,
  3,
  2,
  4,
  5,
  1,
  4,
  4,
  1,
  2,
  5,
  5,
  0,
  4,
  5,
  3],
 [4,
  4,
  1,
  1,
  1,
  5,
  0,
  1,
  2,
  2,
  0,
  3,
  2,
  0,
  4,
  5,
  5,
  3,
  0,
  0,
  5,
  1,
  4,
  1,
  2,
  0,
  5],
 [3,
  0,
  5,
  4,
  1,
  5,
  3,
  1,
  4,
  4,
  5,
  2,
  0,
  1,
  0,
  3,
  1,
  5,
  4,
  5,
  1,
  2,
  4,
  3,
  4,
  1,
  0],
 [1,
  2,
  0,
  1,
  1,
  2,
  4,
  2,
  1,
  3,
  2,
  0,
  0,
  0,
  5,
  3,
  0,
  3,
  1,
  0,
  5,
  2,
  1,
  5,
  4,
  0,
  3],
 [4,
  0,
  0,
  5,
  3,
  2,
  4,
  3,
  4,
  1,
  2,
  5,
  5,
  0,
  4,
  1,
  0,
  3,
  1,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  4],
 [0,
  4,
  5,
  0,
  1,
  0,
  5,
  3,
  1,
  4,
  0,
  2,
  4,
  3,
  0,
  3,
  3,
  1,
  1,
  1,
  0,
  2,
  2,
  1,
  3,
  1,
  1],
 [4,
  5,
  4,
  1,
  3,
  4,
  5,
  3,
  2,
  0,
  0,
  5,
  3,
  5,
  1,
  5,
  4,
  4,
  1,
  5,
  1,
  4,
  1,
  2,
  4,
  2,
  1],
 [4,
  4,
  5,
  5,
  0,
  2,
  3,
  0,
  2,
  4